In [1]:
from utils import BasicDataset, dataSet
from unet import UNet
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.benchmarks.utils import AvalancheDataset, as_classification_dataset
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive,EWC
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torch import optim
import numpy as np

In [2]:
train_dataset = BasicDataset(images_dir=r"D:\workdir\CL_for_ImageSegmentation\experience\data\train\img",
                             mask_dir=r"D:\workdir\CL_for_ImageSegmentation\experience\data\train\label")


100%|██████████| 253/253 [00:02<00:00, 124.38it/s]


In [3]:
from avalanche.benchmarks import benchmark_from_datasets
from avalanche.benchmarks.utils.data_attribute import DataAttribute
import torch

In [4]:
aval_dataset = AvalancheDataset([train_dataset],
                                data_attributes=[
                                                 DataAttribute([0] * len(train_dataset), "targets_task_labels"),
                                                 ])

In [5]:
bm = benchmark_from_datasets(train=[aval_dataset,aval_dataset,aval_dataset,aval_dataset,aval_dataset],
                             test=[aval_dataset,aval_dataset,aval_dataset,aval_dataset,aval_dataset])

In [6]:
print(f"original training samples = {len(bm.train_stream[0].dataset)}")

original training samples = 253


In [7]:
model = UNet(n_channels=3, n_classes=4)

In [8]:

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

In [9]:
epochs = 5
batch_size= 8
learning_rate = 0.00001
weight_decay = 0.00000001
momentum = 0.999
# CREATE THE STRATEGY INSTANCE (NAIVE)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    # accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    # forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    # confusion_matrix_metrics(num_classes=4, save_image=False,stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

from utils.util import SGNaive
cl_strategy = SGNaive(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)




In [10]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in bm.train_stream:
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    # results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
-- >> Start of training phase << --
0it [00:00, ?it/s]torch.Size([8, 3, 447, 513])
100%|██████████| 32/32 [03:00<00:00,  5.63s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53882.7646
	DiskUsage_MB/train_phase/train_stream = 53882.7646
	Loss_Epoch/train_phase/train_stream = 0.8772
	Loss_MB/train_phase/train_stream = 0.5128
	RunningTime_Epoch/train_phase/train_stream = 0.2721
	Time_Epoch/train_phase/train_stream = 180.1530
0it [00:00, ?it/s]torch.Size([8, 3, 447, 513])
100%|██████████| 32/32 [03:00<00:00,  5.65s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream = 53894.5586
	DiskUsage_MB/train_phase/train_stream = 53894.5586
	Loss_Epoch/train_phase/train_stream = 0.3358
	Loss_MB/train_phase/train_stream = 0.2129
	RunningTime_Epoch/train_phase/train_stream = 0.2867
	Time_Epoch/train_phase/train_stream = 180.8538
0it [00:00, ?it/s]torch.Size([8, 3, 447, 513])
100%|██████████| 32/32 [03:00<00:00,  5.65s/it]
Epoch 2 ended.
	DiskUsage_Epoch/

In [ ]:
cl_strategy = EWC(
    model, 
    optimizer,
    CrossEntropyLoss(), 
    0.01,
    train_mb_size=batch_size, 
    train_epochs=epochs, 
    eval_mb_size=batch_size,
    evaluator=eval_plugin)

In [ ]:
cl_strategy.mb_output.shape

In [ ]:
cl_strategy.mb_x.shape

In [ ]:
cl_strategy.mb_y.shape

In [ ]:
cl_strategy.mbatch.shape

In [ ]:
type(experience.dataset.target[0])

In [ ]:
bm.train_stream[0].dataset[0]